Jupyter notebook is useful as we can keep datasets in memory between training runs

## Load datasets:

In [39]:
# Don't cache other files https://stackoverflow.com/a/57245926
%reload_ext autoreload
%autoreload 2

from train import train_model
from torch.utils.data import random_split, ConcatDataset
from face_dataset import *
from label_funcs import *
import preprocessor
from ds_transforms import *

# Define dataset parameters
ds_size = 6667
equalise = False
processor = preprocessor.processor(w=28, h=28)
transform = alexnet_transform(size=28)
equalise = False
augment = False
print_errors = False

# adience_gender_ds = FastDataset(
#     'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\adience',
#     label_func=age_label_all, transform=transform, processor=processor,
#     ds_size=ds_size, print_errors=print_errors, equalise=equalise,
#     augment=augment)

imdb_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\other\\imdb_crop',
    label_func=age_label_all, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)

wiki_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\raw_imdbwiki_crop\\wiki_crop',
    label_func=age_label_all, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)

utkface_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\utkface_itw',
    label_func=age_label_all_utk, transform=transform, processor=processor,
    ds_size=ds_size, print_errors=print_errors, equalise=equalise,
    augment=augment)
    
train_val_set = ConcatDataset([wiki_gender_ds, imdb_gender_ds, utkface_gender_ds])

# Split dataset into training and validation sets
val_split_ratio = 0.2
val_size = int(val_split_ratio * len(train_val_set))
train_size = len(train_val_set) - val_size
train_set, val_set = random_split(train_val_set, [train_size, val_size])
print(f'Split dataset into {len(train_set)} training and {len(val_set)} validation examples')

test_set = val_set

Reading and processing 6667 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\utkface_itw...



6667 items successfully prepared

Split dataset into 6400 training and 1599 validation examples


## Train desired models:

In [41]:

from torch.nn import CrossEntropyLoss, Loss  # classif. -> CEL, regress. -> MSE
from torch.optim import Adam, SGD
from networks import *
import tests

model = train_model(
    model=LeNet(1), 
    train_set=train_set,
    val_set=val_set,
    loss_fn=MSELoss(),
    optim_fn=Adam,
    learning_rate=0.0001,
    patience=3,
    model_save_dir='../models')

tests.mae(model, test_set, print_results=True)


TRAINING MODEL LeNet-1_2604-1524.pt WITH PARAMS:
 - Architecture: LeNet
 - Learning rate: 0.0001
 - Optimizer: Adam
 - Loss function: MSELoss()
 - Other notes: None

+---------------+---------------+---------------+---------------+---------------+
|         EPOCH | EXAMPLES SEEN |    TRAIN LOSS |      VAL LOSS |  ELAPSED TIME |
+---------------+---------------+---------------+---------------+---------------+


|             1 |          6400 |       0.03861 |       0.03138 |          0:17 |


|             2 |         12800 |       0.02916 |       0.02934 |          0:34 |


|             3 |         19200 |       0.02644 |       0.02499 |          0:51 |


|             4 |         25600 |       0.02442 |       0.02412 |          1:11 |


...

| *          14 |         89600 |       0.02134 |       0.02304 |          4:03 |


| **         15 |         96000 |       0.02129 |       0.02328 |          4:21 |


| ***        16 |        102400 |       0.02126 |       0.02303 |          4:37 |
+---------------+---------------+---------------+---------------+---------------+

Halting training - 3 epochs without improvement

Training took 4m 37s (17.0s per epoch)

Best model from session saved to '../models\LeNet-1_2604-1524.pt'

Testing model accuracy...


MAE: 0.12


0.11500172143991927